In [4]:
import librosa
from collections import Counter
from flask import Flask, request, abort, jsonify, send_from_directory
import werkzeug
import keras.models
import numpy as np
import scipy.misc
import wave

RATE = 24000
N_MFCC = 13
COL_SIZE = 30
model = keras.models.load_model('model_5000_1610_2.h5')

def get_wav(f):
    y, sr = librosa.load(f)
    return librosa.core.resample(y=y,orig_sr=sr,target_sr=RATE, scale=True)

def to_mfcc(wav):
    return librosa.feature.mfcc(y=wav, sr=RATE, n_mfcc=N_MFCC)

def segment_one(mfcc):
    segments = []
    for start in range(0, int(mfcc.shape[1] / COL_SIZE)):
        segments.append(mfcc[:, start * COL_SIZE:(start + 1) * COL_SIZE])
    return(np.array(segments))

def predict_class_audio(MFCCs, model):
    MFCCs = MFCCs.reshape(MFCCs.shape[0],MFCCs.shape[1],MFCCs.shape[2],1)
    y_pre = model.predict_proba(MFCCs,verbose=0)
    return y_pre

def cal(y_pre):
    sum_s=0
    sum_j=0;
    for s,j in y_pre:
        sum_s+=s
        sum_j+=j
    t = sum_s+sum_j
    seoul = sum_s/t*100
    jeon = sum_j/t*100
    return seoul,jeon

def predict(f):
    get = get_wav(f)    
    X = to_mfcc(get)
    X = segment_one(X)
    y_pre = predict_class_audio(X, model)
    y0, y1 = cal(y_pre)
    return y0,y1


predict("mv01_t01_s01.wav")

(0.00415801793769892, 99.9958419820623)

In [5]:
cnt=0
app = Flask(__name__)
@app.route('/', methods = ['POST'])
def uploadfile():
    f = request.files['file']
    pcmdata = f.read()
    
    fName='record{}.wav'.format(count())
    with wave.open(fName, 'wb') as wav:
        wav.setparams((1, 2, 16000, 0, 'NONE', 'NONE'))
        wav.writeframes(pcmdata)
    try:
        jeon,seoul = predict(fName)
    except:
        return "녹음이 너무 짧습니다."
    
    result="서울 억양은 {0:0.2f}% \n전라도 억양은 {1:0.2f}% 입니다.".format(seoul,jeon)
    return result

def count():
    global cnt
    cnt = cnt+1
    return cnt

app.run(host="0.0.0.0", port=5000, debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


I0819 09:47:01.732700  4524 _internal.py:122]  * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
I0819 09:47:18.491916  6156 _internal.py:122] 192.168.0.180 - - [19/Aug/2019 09:47:18] "POST / HTTP/1.1" 200 -
I0819 09:47:36.851229 11132 _internal.py:122] 192.168.0.180 - - [19/Aug/2019 09:47:36] "POST / HTTP/1.1" 200 -
I0819 09:48:42.460972 11148 _internal.py:122] 192.168.0.186 - - [19/Aug/2019 09:48:42] "POST / HTTP/1.1" 200 -
I0819 09:50:26.822364  5408 _internal.py:122] 192.168.0.177 - - [19/Aug/2019 09:50:26] "POST / HTTP/1.1" 200 -
I0819 09:51:52.664318 10336 _internal.py:122] 192.168.0.107 - - [19/Aug/2019 09:51:52] "POST / HTTP/1.1" 200 -
I0819 09:56:48.348053  2176 _internal.py:122] 192.168.0.107 - - [19/Aug/2019 09:56:48] "POST / HTTP/1.1" 200 -
I0819 09:56:50.955718   424 _internal.py:122] 192.168.0.107 - - [19/Aug/2019 09:56:50] "POST / HTTP/1.1" 200 -
I0819 10:28:44.011476  6572 _internal.py:122] 192.168.0.186 - - [19/Aug/2019 10:28:44] "POST / HTTP/1.1" 200 -
I0819 10